In [ ]:
#r "nuget: Yueyinqiu.Su.D2lTorchSharp, *-*"

2.1.1. 入门

In [ ]:
using TorchSharp;

In [ ]:
var x = torch.arange(12);
x

In [ ]:
x.shape

In [ ]:
x.numel()

In [ ]:
var X = x.reshape(3, 4);
X

In [ ]:
torch.zeros([2, 3, 4])

In [ ]:
torch.ones([2, 3, 4])

In [ ]:
torch.randn(3, 4)

In [ ]:
torch.tensor(new long[,] {{ 2, 1, 4, 3 }, { 1, 2, 3, 4 }, { 4, 3, 2, 1 }})

2.1.2. 运算符

In [ ]:
var x = torch.tensor(new double[] { 1.0, 2, 4, 8 });
var y = torch.tensor(new long[] { 2, 2, 2, 2 });
(x + y, x - y, x * y, x / y, x.pow(y))

In [ ]:
torch.exp(x)

In [ ]:
var X = torch.arange(12, dtype: torch.float32).reshape([3, 4]);
var Y = torch.tensor(new double[,] {{ 2.0, 1, 4, 3 }, { 1, 2, 3, 4 }, { 4, 3, 2, 1 }});
(torch.cat([X, Y], dim: 0), torch.cat([X, Y], dim: 1))

In [ ]:
X == Y

In [ ]:
X.sum()

2.1.3. 广播机制

In [ ]:
var a = torch.arange(3).reshape([3, 1]);
var b = torch.arange(2).reshape([1, 2]);
(a, b)

In [ ]:
a + b

2.1.4. 索引和切片

In [ ]:
(X[-1], X[1..3])

In [ ]:
X[1, 2] = 9;
X

In [ ]:
X[0..2, ..] = 12;
X

2.1.5. 节省内存

与 Python 不同， C# 不能单独自定义 `+=` 运算。这意味着以下两种做法在 TorchSharp 中不可能有任何区别：

In [ ]:
var before = Y;
Y = Y + X;
object.ReferenceEquals(before, Y)

In [ ]:
var before = Y;
Y += X;
object.ReferenceEquals(before, Y)

要进行原地操作，可以使用带有下划线的相应方法，如 `add_` ：

In [ ]:
var before = Y;
Y.add_(X);
object.ReferenceEquals(before, Y)

切片当然也是可以使用的：

In [ ]:
var Z = torch.zeros_like(Y);
var before = Z;
Z[..] = X + Y;
object.ReferenceEquals(before, Z)

此外，与 PyTorch 不同，在 TorchSharp 中，我们需要额外地关心内存、显存的使用问题，其提供三种主要的[内存管理](https://github.com/dotnet/TorchSharp/wiki/Memory-Management)方式。

第一种方案是依赖 `Tensor` 类型的析构函数，其中会自动释放底层的 LibTorch 张量。这种方式胜在简单，即不用编写任何额外的代码。但由于析构函数是在垃圾回收时触发的，而在内存占用不高时，即使显存占用很高也不一定会触发垃圾回收，导致相关张量始终得不到释放，最终显存溢出。

第二种方案是手动调用张量的 `Dispose` 方法，或者使用 `using` 语句。这种方法看起来美好，但实际用起来非常繁琐。举例而言，仅仅是计算 `p = m + n + o;` ，为正确释放 `m + n` 的结果，必须改用 `using var temp = m + n; p = temp + o;` 。而这种情况常常大量出现在我们的代码中，因此这种方案并没有想象中那么漂亮。

因此我们有第三种方案，即 `torch.NewDisposeScope` 。在 `Tensor` 对象被创建时，会自动注册到其附近的 `DisposeScope` ，并随着 `DisposeScope` 的释放一同释放。通常我们会使用 `DisposeScope` 包裹一个小批量的训练过程，从而释放这一轮计算中得到的所有张量（因为下一轮开始就不会再用到它们了）。由于 `DisposeScope` 是 TorchSharp 额外提供的功能，或许我们需要通过一些例子来说明它：


In [ ]:
var disposeScope = torch.NewDisposeScope();

var m = torch.zeros([]) + 1;
var n = torch.zeros([]) + 2;
var l = torch.zeros([]) + 3;
var p = m + n + l;
Console.WriteLine((double)p);

disposeScope.Dispose();

try
{
    Console.WriteLine((double)p);
}
catch (Exception ex)
{ 
    Console.WriteLine(ex.Message);
}

当然，事实上我们会使用 `using` 而不是手动调用 `disposeScope.Dispose` 。

由于在 `DisposeScope` 中创建的张量都会随着它一同释放，因此当我们需要某个张量作为函数返回值的时候，可能需要把它移出函数内的 `DisposeScope` ，下面是一个相关的示例：

In [ ]:
torch.Tensor MyTensorSum(torch.Tensor m, torch.Tensor n, torch.Tensor o)
{
    using (torch.NewDisposeScope())
    {
        var p = m + n + o;
        return p.MoveToOuterDisposeScope();
    }
    // 只有中间值 m + n 会在这里被释放。
    // m 、 n 、 o 都是在外界创建的，与此 DisposeScope 无关。
    // 而 p 已被移动到更外层的 DisposeScope （此处没有更外层，即最终它不关联任何 DisposeScope ）。
}

var m = torch.zeros([]) + 1;
var n = torch.zeros([]) + 2;
var l = torch.zeros([]) + 3;
var p = MyTensorSum(m, n, l);
Console.WriteLine((double)p);

2.1.6. 转换为其他Python对象

In [ ]:
var A = X.data<float>().ToNDArray();
var B = torch.tensor((float[,])A);
(A.GetType(), B.GetType())

In [ ]:
var a = torch.tensor(new double[] { 3.5 });
(a, a.item<double>(), (double)a, (long)a)

此外，其与字符串的转换也值得一提，因为它和 PyTorch 的表现不太一致。特别是，其默认使用 `TensorStringStyle.Metadata` ，不包含张量中的元素，可能带来误导。（例如你可能把一个标量误以为是空的张量，因为它只打印 `[], type = Float64, device = cpu` 。）更多相关信息可以参考[此处](https://github.com/dotnet/TorchSharp/wiki/Tensor-String-Formatting)。

In [ ]:
Console.WriteLine($"ToString(): {a.ToString()}");
Console.WriteLine($"ToString(): {a.ToString(TensorStringStyle.Julia)}");
Console.WriteLine($"ToString(): {a.cstr()}");